# Lectura de archivos y gráficación de datos

El archivo `BACHOCOB.MX.2019-21.csv` contiene datos de cotizaciones de una acción de la Bolsa Mexicana de Valores. 

Éstas se pueden leer abriendo el archivo con `open` y recuperando cada línea con `readline`.

`open` lleva dos parámetros: el nombre del archivo a abrir (con todo y su ruta de acceso) y el _modo_ como se va a abrir. Hoy usaremos `'r'` para el modo, que significa abrir en modo lectura (_read_).

In [ ]:
# Nombre el archivo
filename = 'BACHOCOB.MX.2019-21.csv'
# Abrir el archivo
with open(filename, 'r') as f:
    # Imprimir las primeras cinco líneas
    for i in range(5):
        print(f.readline())

`readline` lee una línea del archivo (con todo y el carácter de fin de línea, ¿se fijaron en el espaciado extra del `print`?)

El archivo es un archivo de texto (en formato CSV: _comma-separated values_) y la primera línea contiene el encabezado. Son siete columnas:

- Date
- Open
- High
- Low
- Close
- Adj Close
- Volume

Vamos a separar los datos:

In [ ]:
lineas = []
with open(filename, 'r') as f:
    # Leer las primeras cinco líneas
    for i in range(5):
        li = f.readline().strip()          # Quitarle el carácter EOL
        lineas.append(li)                  # Agregarlo a la lista
lineas

Hasta aquí, tenemos una lista (`lineas`) con cada una de las líneas del archivo, incluyendo el encabezado.

Vamos a separarlo en siete listas homogéneas (cada lista contiene el mismo tipo de información), una para cada columna.

In [ ]:
fecha = []
apertura = []
alto = []
bajo = []
cierre = []
cierre_aj = []
volumen = []

for li in lineas[1:]:                  # Sin encabezados
    li = li.split(',')                 # Separar por las comas
    # Desempacar
    fecha.append(li[0])
    apertura.append(float(li[1]))
    alto.append(float(li[2]))
    bajo.append(float(li[3]))
    cierre.append(float(li[4]))
    cierre_aj.append(float(li[5]))
    volumen.append(float(li[6]))
    
# Imprimir los datos
print("Fecha       Apertura      Alto      Bajo    Cierre    C. Aj.   Volumen")
for i in range(len(fecha)):
    print(f'{fecha[i]:10}'
          f'{apertura[i]:10.4f}'
          f'{alto[i]:10.4f}'
          f'{bajo[i]:10.4f}'
          f'{cierre[i]:10.4f}'
          f'{cierre_aj[i]:10.4f}'
          f'{volumen[i]:10,.0f}')

Hicimos siete listas homogéneas. Pudimos haber hecho una lista de listas u otra estructura de Python, como un diccionario, etc.

Esta sería una forma de trabajar con un archivo CSV utilizando únicamente la funcionalidad nativa de Python que hemos estudiado. 

## Uso de `pandas`

Sin embargo, no tenemos por qué limitarnos a dicha funcionalidad nativa; existen una infinidad de módulos de terceros que proporcionan funcionalidad adicional. 

A manera de ejemplo, vamos a utilizar la popular biblioteca `pandas`, que incluye, entre muchas otras cosas, la funcionalidad de leer archivos CSV.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

cotizaciones = pd.read_csv(filename)
cotizaciones.head()

Más bonito y más fácil, ¿no?

La estructura que regresa la función `read_csv` es un `DataFrame`, que es la estructura básica de `pandas`. Está compuesta por series (tipo `Series`) de datos organizadas en columnas.

La columna `Date` es tipo `str`. La vamos a convertir a tipo fecha (`datetime`) para su mejor interpretación a la hora de graficar.

In [ ]:
cotizaciones.Date = pd.to_datetime(cotizaciones.Date)
cotizaciones.head()

## Graficar las cotizaciones

Las cotizaciones al cierre se pueden graficar de manera sencilla utilizando la biblioteca `matplotlib`.

In [ ]:
# Inicializar los elementos de la gráfica
fig, ax = plt.subplots()
simbolo = 'BACHOCOB.MX'
fecha = cotizaciones.Date             # Dos formas equivalentes de
cierre = cotizaciones['Close']        # referirse a las columnas
ax.plot(fecha, cierre)
ax.set(title=simbolo)
plt.show()

## Cálculo de los rendimientos logarítmicos

Las series de `pandas`, por estar basadas en vectores de `numpy`, permiten realizar operaciones con todos sus elementos al mismo tiempo, sin necesidad de utilizar ciclos para hacer la operación con cada uno de ellos. Esto hace posible el cálculo de los rendimientos logarítmicos de manera muy sencilla.

In [ ]:
rend = np.log(cierre / cierre.shift())
rend.head()

El cálculo de los rendimientos logarítmicos se lleva a cabo de acuerdo a la fórmula:

$$R = \ln(\frac{V_f}{V_i})$$

donde $V_i$ es el valor inicial y $V_f$ es el valor final.

En la fórmula de Python, la división es la cotización al cierre del día entre la cotización al cierre del día anterior, como las columnas de cotización están ordenadas por fecha ascendente, el cierre del día anterior se encuentra en el renglón de arriba, por eso se usa `shift` para alinearlo y poder hacer la división.

El primer renglón no tiene cotización anterior, por eso el cálculo regresa `NaN` (_not a number_).

`subplots` nos permite especificar varias gráficas en la misma figura indicando el número de renglones y columnas. Regresa un objeto `Axes` para cada gráfica de la cuadrícula.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True)
ax1.plot(fecha, cierre, label='Cierre')
ax1.set(title=simbolo)
ax1.legend()
ax2.plot(fecha, rend, label='Rendimientos')
ax2.legend()
plt.show()

## Calcular la volatilidad

Añadiremos ahora la gráfica de la volatilidad. Primero, calculamos la desviación estándar móvil sobre el último año (252 días).

In [ ]:
volatilidad = cierre.rolling(252).std()

Utilizamos de nuevo `subplots` para apilar las tres gráficas en la misma figura.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex=True)
ax1.plot(fecha, cierre, label='Cierre')
ax1.set(title=simbolo)
ax1.legend()
ax2.plot(fecha, rend, label='Rendimientos')
ax2.legend()
ax3.plot(fecha, volatilidad, label='Volatilidad (252 días)')
ax3.legend()
fig.autofmt_xdate()
plt.show()

## Graficar medias móviles y volumen de operación

Finalmente, se generará dos gráficas. La primera mostrará las cotizaciones al cierre junto con las medias móviles a 42 y 252 días; la segunda, el volumen de operación. Se desea que la primera gráfica ocupe una mayor proporción del área de la ventana que la segunda.

Primero se calculan las medias móviles de manera similar a como se calcularon las volatilidades más arriba.

In [ ]:
media42d = cierre.rolling(42).mean()
media252d = cierre.rolling(252).mean()

Y se grafican los datos.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, gridspec_kw={'height_ratios': [3, 1]})
ax1.plot(fecha, cierre, label='Cierre')
ax1.plot(fecha, media42d, label='Media móvil (42 días)')
ax1.plot(fecha, media252d, label='Media móvil (252 días)')
ax1.set(title=simbolo)
ax1.legend()
ax2.bar(fecha, cotizaciones.Volume, label='Volumen')
ax.legend()
fig.autofmt_xdate()
plt.show()

![pandas](https://pandas.pydata.org/static/img/pandas.svg "pandas")

Documentación de `pandas`: https://pandas.pydata.org/

![numpy](https://numpy.org/neps/_static/numpylogo.svg "numpy")

Documentación de `numpy`: https://numpy.org/

![matplotlib](https://matplotlib.org/stable/_static/logo2_compressed.svg "matplotlib")

Documentación de `matplotlib`: https://matplotlib.org/ 

Galería de ejemplos de `matplotlib`: https://matplotlib.org/stable/gallery/index.html